# Exploratory Data Analysis
Analysis of UK National Grid demand data from NESO (National Energy System Operator).

In [68]:
# Imports
import requests
from bs4 import BeautifulSoup
from pathlib import Path
import pandas as pd
import hashlib
from datetime import datetime, timedelta
from supabase import create_client, Client
import os
from dotenv import load_dotenv

load_dotenv()

True

## Demand Data

| Column name                 | Description                                                                                              |
| --------------------------- | -------------------------------------------------------------------------------------------------------- |
| `SETTLEMENT_DATE`           | Calendar date of the electricity settlement period (UTC).                                                |
| `SETTLEMENT_PERIOD`         | Half-hour settlement period within the day (1–48).                                                       |
| `FORECAST_ACTUAL_INDICATOR` | Indicates whether the data point is **Actual (A)** or **Forecast (F)**.                                  |
| `ND`                        | **National Demand** – estimated total GB electricity demand including embedded generation.               |
| `TSD`                       | **Transmission System Demand** – demand seen by the transmission network (excludes embedded generation). |
| `ENGLAND_WALES_DEMAND`      | Electricity demand specific to England and Wales.                                                        |
| `EMBEDDED_WIND_GENERATION`  | Electricity generated by wind connected to distribution networks (not transmission).                     |
| `EMBEDDED_WIND_CAPACITY`    | Installed capacity of embedded wind generation.                                                          |
| `EMBEDDED_SOLAR_GENERATION` | Electricity generated by embedded solar PV.                                                              |
| `EMBEDDED_SOLAR_CAPACITY`   | Installed capacity of embedded solar PV.                                                                 |
| `NON_BM_STOR`               | Non-Balancing Mechanism storage output (e.g. small-scale batteries).                                     |
| `PUMP_STORAGE_PUMPING`      | Electricity demand used to pump water into pumped-storage hydro (negative net generation).               |
| `SCOTTISH_TRANSFER`         | Net electricity transfer between Scotland and England/Wales.                                             |
| `IFA_FLOW`                  | Power flow on the **IFA** interconnector (GB–France).                                                    |
| `IFA2_FLOW`                 | Power flow on the **IFA2** interconnector (GB–France).                                                   |
| `BRITNED_FLOW`              | Power flow on the **BritNed** interconnector (GB–Netherlands).                                           |
| `MOYLE_FLOW`                | Power flow on the **Moyle** interconnector (GB–Northern Ireland).                                        |
| `EAST_WEST_FLOW`            | Power flow on the **East-West** interconnector (GB–Ireland).                                             |
| `NEMO_FLOW`                 | Power flow on the **NEMO** interconnector (GB–Belgium).                                                  |
| `NSL_FLOW`                  | Power flow on the **North Sea Link** interconnector (GB–Norway).                                         |
| `ELECLINK_FLOW`             | Power flow on the **ElecLink** interconnector (GB–France).                                               |
| `VIKING_FLOW`               | Power flow on the **Viking Link** interconnector (GB–Denmark).                                           |
| `GREENLINK_FLOW`            | Power flow on the **Greenlink** interconnector (GB–Ireland).                                             |       


### Download Historic Data
Download CSV files for each year (2020-2025) from NESO data portal.

In [ ]:
# Supabase connection
supabase_url: str = os.getenv("SUPABASE_URL")
supabase_key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)
print("Connected to Supabase.")

In [53]:
# Configuration
BASE_PAGE = "https://www.neso.energy/data-portal/historic-demand-data/historic_demand_data_{}"
YEARS = range(2020, 2026)
OUT_DIR = Path("neso_historic_demand")
OUT_DIR.mkdir(exist_ok=True)

In [54]:
# HTTP session with custom User-Agent
session = requests.Session()
session.headers.update({"User-Agent": "academic-dashboard-project"})

In [55]:
# Helper function
def sha256(path: Path) -> str:
    """Calculate SHA256 hash of a file for deduplication."""
    h = hashlib.sha256()
    with open(path, "rb") as f:
        for chunk in iter(lambda: f.read(8192), b""):
            h.update(chunk)
    return h.hexdigest()

In [57]:
# Define possible date formats globally
date_formats = [
    '%Y-%m-%d',        # 2025-06-27
    '%d-%b-%y',        # 26-Oct-23
    '%d-%b-%Y',        # 26-Oct-2023
    '%d-%B-%y',        # 26-October-23
    '%d-%B-%Y',        # 26-October-2023
    '%Y-%m-%dT%H:%M:%S.%fZ',
    '%Y-%m-%dT%H:%M:%S',
]

In [58]:
def standardize_date(date_val):
    """Convert various date formats to a standard datetime object."""
    if pd.isna(date_val):
        return pd.NaT
    
    date_str = str(date_val).strip().upper()
    
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_str, format=fmt.upper() if '%b' in fmt or '%B' in fmt else fmt)
        except (ValueError, TypeError):
            continue
    
    try:
        return pd.to_datetime(date_str, dayfirst=True)
    except:
        return pd.NaT

In [59]:
# Download and process data for each year
all_dfs = []
seen_hashes = set()

for year in YEARS:
    print(f"Processing {year}...")
    page_url = BASE_PAGE.format(year)

    # Fetch dataset page
    r = session.get(page_url, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")

    # Find CSV download link
    csv_url = None
    for a in soup.find_all("a", href=True):
        href = a["href"].lower()
        if "download" in href and href.endswith(".csv"):
            csv_url = a["href"]
            break

    if not csv_url:
        print(f"  No CSV found for {year}, skipping...")
        continue

    if csv_url.startswith("/"):
        csv_url = "https://www.neso.energy" + csv_url

    csv_path = OUT_DIR / f"historic_demand_{year}.csv"

    # Download CSV
    with session.get(csv_url, stream=True, timeout=60) as resp:
        resp.raise_for_status()
        with open(csv_path, "wb") as f:
            for chunk in resp.iter_content(8192):
                f.write(chunk)

    # Verify uniqueness
    file_hash = sha256(csv_path)
    if file_hash in seen_hashes:
        print(f"  Duplicate detected for {year}, skipping...")
        continue
    seen_hashes.add(file_hash)

    # Load and standardize date format
    df_year = pd.read_csv(csv_path)
    df_year['SETTLEMENT_DATE'] = df_year['SETTLEMENT_DATE'].apply(standardize_date)
    df_year["SOURCE_YEAR"] = year

    all_dfs.append(df_year)
    print(f"  Loaded {len(df_year)} records ({df_year['SETTLEMENT_DATE'].min().date()} to {df_year['SETTLEMENT_DATE'].max().date()})")

# Combine all years
df = pd.concat(all_dfs, ignore_index=True)
print(f"\nTotal records: {len(df)}")

Processing 2020...
  Loaded 17568 records (2020-01-01 to 2020-12-31)
Processing 2021...
  Loaded 17520 records (2021-01-01 to 2021-12-31)
Processing 2022...
  Loaded 17520 records (2022-01-01 to 2022-12-31)
Processing 2023...
  Loaded 17520 records (2023-01-01 to 2023-12-31)
Processing 2024...
  Loaded 17568 records (2024-01-01 to 2024-12-31)
Processing 2025...
  Loaded 16800 records (2025-01-01 to 2025-12-16)

Total records: 104496


In [60]:
# Preview the data
print(f"Columns: {df.columns.tolist()}")
print(f"Date range: {df['SETTLEMENT_DATE'].min()} to {df['SETTLEMENT_DATE'].max()}")
df.sample(5)

Columns: ['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD', 'ND', 'TSD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION', 'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING', 'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW', 'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'VIKING_FLOW', 'GREENLINK_FLOW', 'SOURCE_YEAR', 'SCOTTISH_TRANSFER']
Date range: 2020-01-01 00:00:00 to 2025-12-16 00:00:00


,SETTLEMENT_DATE,SETTLEMENT_PERIOD,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,...,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,VIKING_FLOW,GREENLINK_FLOW,SOURCE_YEAR,SCOTTISH_TRANSFER
34286,2021-12-15,15,32405,35654,30075,2568,6527,0,14001,0,...,108,375,369,-587,693,0,0,0,2021,NaN
80737,2024-08-09,4,17125,21825,16386,3448,6563,0,18053,0,...,0,-301,-526,-958,-429,159,0,0,2024,3247.0
90348,2025-02-25,13,26637,30627,25087,2087,6606,0,19726,0,...,-1071,-131,0,-632,1397,850,-1094,-439,2025,4377.0
32542,2021-11-08,47,24344,25771,23057,2649,6527,0,13915,0,...,924,122,-65,584,693,0,0,0,2021,NaN
22657,2021-04-17,4,24955,26196,22744,382,6527,0,13653,0,...,0,-19,0,975,0,0,0,0,2021,NaN


### Data Cleaning
Remove unnecessary columns and create a unified datetime column.

In [64]:
# Remove unnecessary columns
columns_to_remove = ['_id', '_full_text', '_count', 'rank', 'FORECAST_ACTUAL_INDICATOR', 'SOURCE_YEAR']
df_cleaned = df.drop(columns=[col for col in columns_to_remove if col in df.columns], errors='ignore')

print(f"Columns after cleaning: {df_cleaned.columns.tolist()}")

Columns after cleaning: ['SETTLEMENT_DATE', 'SETTLEMENT_PERIOD', 'ND', 'TSD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION', 'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING', 'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW', 'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'VIKING_FLOW', 'GREENLINK_FLOW', 'SCOTTISH_TRANSFER']


In [69]:
# Create unified DATETIME column from SETTLEMENT_DATE and SETTLEMENT_PERIOD
df_cleaned['DATETIME'] = df_cleaned.apply(
    lambda row: row['SETTLEMENT_DATE'] + timedelta(minutes=(int(row['SETTLEMENT_PERIOD']) - 1) * 30),
    axis=1
)

In [70]:
# Reorder columns and remove original date columns
cols = ['DATETIME'] + [col for col in df_cleaned.columns if col not in ['DATETIME', 'SETTLEMENT_DATE', 'SETTLEMENT_PERIOD']]
df_cleaned = df_cleaned[cols]

In [71]:

print(f"Final shape: {df_cleaned.shape}")
print(f"Date range: {df_cleaned['DATETIME'].min()} to {df_cleaned['DATETIME'].max()}")
df_cleaned.head()

Final shape: (104496, 21)
Date range: 2020-01-01 00:00:00 to 2025-12-16 23:30:00


,DATETIME,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,...,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,VIKING_FLOW,GREENLINK_FLOW,SCOTTISH_TRANSFER
0,2020-01-01 00:00:00,26340,27153,23821,1073,6465,0,13040,0,15,...,0,852,-151,-47,854,0,0,0,0,NaN
1,2020-01-01 00:30:00,26921,27684,24393,1020,6465,0,13040,0,17,...,0,853,-146,0,854,0,0,0,0,NaN
2,2020-01-01 01:00:00,26569,27240,24085,1010,6465,0,13040,0,18,...,0,852,-53,0,854,0,0,0,0,NaN
3,2020-01-01 01:30:00,25754,26435,23350,1043,6465,0,13040,0,15,...,0,852,-66,0,854,0,0,0,0,NaN
4,2020-01-01 02:00:00,25075,25824,22788,1001,6465,0,13040,0,15,...,0,853,-74,-60,854,0,0,0,0,NaN


## Data Quality

Analyse the data quality

In [80]:
# Quick overview
print(f"{len(df_cleaned):,} records")
print(f"From {df_cleaned['DATETIME'].min()} to {df_cleaned['DATETIME'].max()}")
print(f"\n{len(df_cleaned.columns)} columns: {df_cleaned.columns.tolist()}")

104,496 records
From 2020-01-01 00:00:00 to 2025-12-16 23:30:00

21 columns: ['DATETIME', 'ND', 'TSD', 'ENGLAND_WALES_DEMAND', 'EMBEDDED_WIND_GENERATION', 'EMBEDDED_WIND_CAPACITY', 'EMBEDDED_SOLAR_GENERATION', 'EMBEDDED_SOLAR_CAPACITY', 'NON_BM_STOR', 'PUMP_STORAGE_PUMPING', 'IFA_FLOW', 'IFA2_FLOW', 'BRITNED_FLOW', 'MOYLE_FLOW', 'EAST_WEST_FLOW', 'NEMO_FLOW', 'NSL_FLOW', 'ELECLINK_FLOW', 'VIKING_FLOW', 'GREENLINK_FLOW', 'SCOTTISH_TRANSFER']


In [83]:
# Detect Missing values
missing = df_cleaned.isnull().sum()
missing = missing[missing > 0]

if len(missing) > 0:
    print("Missing values:")
    print(missing)
else:
    print("No nulls detected")

Missing values:
SCOTTISH_TRANSFER    52608
dtype: int64


In [84]:
# Detect duplicates
dupes = df_cleaned[df_cleaned.duplicated(subset=['DATETIME'], keep=False)]
print(f"{len(dupes)} duplicate timestamps")

if len(dupes) > 0:
    dupes.sort_values('DATETIME').head(10)

24 duplicate timestamps


In [86]:
# Remove duplicates 
df_cleaned = df_cleaned.drop_duplicates(subset=['DATETIME'], keep='first').reset_index(drop=True)

In [88]:
# Make sure duplicates are dropped 
dupes = df_cleaned[df_cleaned.duplicated(subset=['DATETIME'], keep=False)]
print(f"{len(dupes)} duplicate timestamps")

if len(dupes) > 0:
    dupes.sort_values('DATETIME').head(10)

0 duplicate timestamps


In [90]:
# Check for gaps
df_sorted = df_cleaned.sort_values('DATETIME')
time_diffs = df_sorted['DATETIME'].diff()
gaps = time_diffs[time_diffs > timedelta(minutes=30)]

print(f"Found {len(gaps)} gaps in the time series")

if len(gaps) > 0:
    # show the biggest ones
    gap_df = pd.DataFrame({
        'starts_after': df_sorted.loc[gaps.index - 1, 'DATETIME'].values,
        'duration': gaps.values
    }).sort_values('duration', ascending=False)
    gap_df.head(10)

Found 6 gaps in the time series


In [91]:
# Stats
df_cleaned.describe()

,DATETIME,ND,TSD,ENGLAND_WALES_DEMAND,EMBEDDED_WIND_GENERATION,EMBEDDED_WIND_CAPACITY,EMBEDDED_SOLAR_GENERATION,EMBEDDED_SOLAR_CAPACITY,NON_BM_STOR,PUMP_STORAGE_PUMPING,...,IFA2_FLOW,BRITNED_FLOW,MOYLE_FLOW,EAST_WEST_FLOW,NEMO_FLOW,NSL_FLOW,ELECLINK_FLOW,VIKING_FLOW,GREENLINK_FLOW,SCOTTISH_TRANSFER
count,104484,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,...,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,104484.000000,51882.000000
mean,2022-12-24 11:59:25.165958400,26752.066833,28717.435081,24515.319637,1884.021745,6549.374163,1583.656388,15890.782924,0.421529,192.440192,...,251.905842,276.472618,-144.414886,-94.521946,409.267141,590.256030,220.793031,115.522893,-51.194355,1810.682877
min,2020-01-01 00:00:00,12803.000000,15297.000000,12040.000000,125.000000,6465.000000,0.000000,13040.000000,0.000000,0.000000,...,-1030.000000,-1093.000000,-505.000000,-585.000000,-1024.000000,-1455.000000,-1028.000000,-1465.000000,-539.000000,-2851.000000
25%,2021-06-28 06:22:30,21879.000000,24206.000000,20047.000000,910.000000,6527.000000,0.000000,13721.000000,0.000000,7.000000,...,-2.000000,-104.000000,-444.000000,-374.000000,0.000000,0.000000,0.000000,0.000000,0.000000,281.250000
50%,2022-12-24 11:45:00,25866.000000,27788.000000,23665.000000,1552.000000,6545.000000,6.000000,15029.000000,0.000000,11.000000,...,0.000000,346.000000,-207.000000,0.000000,667.000000,693.000000,0.000000,0.000000,0.000000,1664.000000
75%,2024-06-20 18:07:30,30673.000000,32349.250000,28132.000000,2622.000000,6562.000000,2540.250000,17714.000000,0.000000,157.000000,...,941.000000,917.000000,81.000000,0.000000,963.000000,1395.000000,871.000000,0.000000,0.000000,3223.000000
max,2025-12-16 23:30:00,46433.000000,47760.000000,42458.000000,5962.000000,6622.000000,14035.000000,20993.000000,481.000000,1869.000000,...,1016.000000,1080.000000,499.000000,504.000000,1020.000000,1419.000000,1002.000000,1436.000000,506.000000,6704.000000
std,NaN,6219.736766,5932.001775,5693.937458,1213.376734,38.492120,2484.195058,2550.415513,10.005025,362.019887,...,624.826686,656.898443,281.493949,302.518871,625.839242,702.943795,553.844066,493.307370,162.808656,1854.728008


In [92]:
# Outliers - using IQR on the main demand columns
for col in ['ND', 'TSD', 'ENGLAND_WALES_DEMAND']:
    if col not in df_cleaned.columns:
        continue
    q1, q3 = df_cleaned[col].quantile([0.25, 0.75])
    iqr = q3 - q1
    outliers = df_cleaned[(df_cleaned[col] < q1 - 1.5*iqr) | (df_cleaned[col] > q3 + 1.5*iqr)]
    print(f"{col}: {len(outliers)} outliers ({len(outliers)/len(df_cleaned)*100:.1f}%)")

ND: 237 outliers (0.2%)
TSD: 509 outliers (0.5%)
ENGLAND_WALES_DEMAND: 207 outliers (0.2%)


In [93]:
# How complete is each year? (expecting 48 periods/day × 365 days)
expected = 48 * 365

by_year = df_cleaned.groupby(df_cleaned['DATETIME'].dt.year).size()
completeness = pd.DataFrame({
    'records': by_year,
    'expected': expected,
    'pct': (by_year / expected * 100).round(1)
})
completeness

,records,expected,pct
DATETIME,,,
2020,17566,17520,100.3
2021,17518,17520,100.0
2022,17518,17520,100.0
2023,17518,17520,100.0
2024,17566,17520,100.3
2025,16798,17520,95.9


In [95]:
# Summary
nulls = df_cleaned.isnull().sum().sum()
dupes = df_cleaned.duplicated(subset=['DATETIME']).sum()

print(f"Records: {len(df_cleaned):,}")
print(f"Nulls: {nulls}")
print(f"Dupelicates: {dupes}")
print(f"Range: {df_cleaned['DATETIME'].min().date()} → {df_cleaned['DATETIME'].max().date()}")

Records: 104,484
Nulls: 52602
Dupelicates: 0
Range: 2020-01-01 → 2025-12-16


## Upload to Supabase

Uplaod the cleaned data to the remote database

In [ ]:
# Load environment variables
load_dotenv()

True

In [97]:
# Define keys and client
supabase_url: str = os.getenv("SUPABASE_URL")
supabase_key: str = os.getenv("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)
print("Connected to Supabase.")

Connected to Supabase.


In [ ]:
import numpy as np
import json

# Prep for upload
df_upload = df_cleaned.copy()
df_upload['DATETIME'] = df_upload['DATETIME'].dt.strftime('%Y-%m-%dT%H:%M:%S')
df_upload.columns = df_upload.columns.str.lower()

# Replace NaN and inf with None
df_upload = df_upload.replace([np.nan, np.inf, -np.inf], None)

# Batch upload
batch_size = 1000
for start in range(0, len(df_upload), batch_size):
    end = min(start + batch_size, len(df_upload))
    batch = df_upload.iloc[start:end].to_dict(orient='records')
    
    # Extra cleanup - convert any remaining nan to None
    for record in batch:
        for key, val in record.items():
            if isinstance(val, float) and (np.isnan(val) or np.isinf(val)):
                record[key] = None
    
    response = supabase.table('historic_demand').insert(batch).execute()
    print(f"Uploaded {start} to {end}")

ValueError: Out of range float values are not JSON compliant: nan